In [9]:
#Preprocessing Images for tensorflow CNN model
import os
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

#Define dimensions for images
img_height = 227
img_width = 227
batch_size = 50

img_dir = "C:\\Coding\\Github\\Project_3DPrintingApp\\CNNBuilding\\images_combined"

# Get list of all image file paths
all_image_paths = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]

# Get list of all image labels (assuming filenames are label_image_number.jpg)
all_image_labels = [fname.split('_')[0] for fname in os.listdir(img_dir)]

In [8]:
# Split data into training, validation, and test sets
train_files, test_files, train_labels, test_labels = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=42)
train_files, val_files, train_labels, val_labels = train_test_split(train_files, train_labels, test_size=0.2, random_state=42)

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))

In [10]:
# Function to preprocess images
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img, label

In [11]:
# Apply preprocessing and batching
train_ds = train_ds.map(preprocess_image).batch(batch_size)
val_ds = val_ds.map(preprocess_image).batch(batch_size)
test_ds = test_ds.map(preprocess_image).batch(batch_size)

In [13]:
#Defining tensor flow model
class TensorFlowModel(tf.keras.Model):
    def __init__(self, num_classes=4):
        super(TensorFlowModel, self).__init__()
        self.layer1 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(96, kernel_size=11, strides=4, padding='valid'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)
        ])
        self.layer2 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(256, kernel_size=5, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)
        ])
        self.layer3 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(384, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU()
        ])
        self.layer4 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(384, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU()
        ])
        self.layer5 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)
        ])
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(4096),
            tf.keras.layers.ReLU()
        ])
        self.fc1 = tf.keras.Sequential([
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(4096),
            tf.keras.layers.ReLU()
        ])
        self.fc2 = tf.keras.layers.Dense(num_classes)

    def call(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.flatten(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [15]:
model = TensorFlowModel()


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9256515647849185323
xla_global_id: -1
]
